In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio

%matplotlib inline

In [2]:

wow = '../Data/wowc.mp3'
finished=Audio(url = wow, autoplay = True)

In [3]:
endpoint = 'https://statsapi.web.nhl.com/api/v1/teams/'

In [4]:
args = '?teamId=1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,52,53&expand=team.roster&season=20132014'

In [5]:
all_teams = endpoint + args

In [6]:
response = requests.get(all_teams)
finished

In [7]:
response

<Response [200]>

In [8]:
res = response.json()


In [9]:
res['teams'][0]['roster']['roster'][0]['person']['id']

8448208

In [10]:
allteamplayers = pd.DataFrame()

for team in res['teams']:
    team_name = team['name']
    for person in team['roster']['roster']:
        per ={}
        per['id']=(person['person']['id'])
        per['name']=(person['person']['fullName'])
        per['position']=(person['position']['type'])
        per['team']=team_name
        allteamplayers= allteamplayers.append(per,ignore_index=True)
finished        

In [11]:
allteamplayers

,id,name,position,team
0,8448208.0,Jaromir Jagr,Forward,New Jersey Devils
1,8460542.0,Patrik Elias,Forward,New Jersey Devils
2,8460626.0,Bryce Salvador,Defenseman,New Jersey Devils
3,8464977.0,Dainius Zubrus,Forward,New Jersey Devils
4,8467545.0,Michael Ryder,Forward,New Jersey Devils
...,...,...,...,...
1053,8476460.0,Mark Scheifele,Forward,Winnipeg Jets
1054,8476885.0,Jacob Trouba,Defenseman,Winnipeg Jets
1055,8471219.0,Al Montoya,Goalie,Winnipeg Jets
1056,8471715.0,Ondrej Pavelec,Goalie,Winnipeg Jets


In [12]:
allteamplayers['id']= allteamplayers['id'].astype(int).astype(str)



In [13]:
for i,row in allteamplayers.iterrows():
    endpointall ='https://statsapi.web.nhl.com/api/v1/people/' + str(row['id']) + '/stats/?stats=statsSingleSeason&season=20132014'
    responseall=requests.get(endpointall)
    resall = responseall.json()
    for team in resall['stats']:
        for split in team['splits']:
            if row['position']=='Goalie':
                allteamplayers.at[i,'assists']= 0
                allteamplayers.at[i,'games']= split['stat']['games']
                allteamplayers.at[i,'games_started']= split['stat']['gamesStarted'] 
                allteamplayers.at[i,'goals']= 0
                allteamplayers.at[i,'time_on_ice']= split['stat']['timeOnIcePerGame']
                allteamplayers.at[i,'blocked']= 0
                allteamplayers.at[i,'penaltymin']= 0
                allteamplayers.at[i,'plusMinus']= 0
                allteamplayers.at[i,'wins']= split['stat']['wins']
                allteamplayers.at[i,'losses']= split['stat']['losses']
                allteamplayers.at[i,'gaa']= split['stat']['goalAgainstAverage']
                allteamplayers.at[i,'shots_against']= split['stat']['shotsAgainst']
                allteamplayers.at[i,'goals_against']= split['stat']['goalsAgainst']
                allteamplayers.at[i,'shutouts']= split['stat']['shutouts']
                allteamplayers.at[i,'yearid']= '2013-2014'
            else: 
                allteamplayers.at[i,'assists']= split['stat']['assists']
                allteamplayers.at[i,'games']= split['stat']['games']
                allteamplayers.at[i,'games_started']= 0
                allteamplayers.at[i,'goals']= split['stat']['goals']
                allteamplayers.at[i,'time_on_ice']= split['stat']['timeOnIcePerGame']
                allteamplayers.at[i,'blocked']= split['stat']['blocked']
                allteamplayers.at[i,'penaltymin']= split['stat']['pim']
                allteamplayers.at[i,'plusMinus']= split['stat']['plusMinus']
                allteamplayers.at[i,'wins']= 0
                allteamplayers.at[i,'losses']= 0
                allteamplayers.at[i,'gaa']= 0
                allteamplayers.at[i,'shots_against']= 0
                allteamplayers.at[i,'goals_against']= 0
                allteamplayers.at[i,'shutouts']= 0
                allteamplayers.at[i,'yearid']= '2013-2014'
finished

In [14]:
allteamplayers.shape

(1058, 19)

In [15]:
allteamplayers.to_csv('../Data/2013_2014_stats.csv')

In [16]:
#this process was repeated for all 10 seasons 2011/12-2020/21